In [1]:
import pandas as pd
import datetime as dt
import numpy as np

pd.options.mode.chained_assignment = None  # default='warn'

# Consigna

Vamos a trabajar con una base de datos elaborada por [Sysarmy](https://sysarmy.com/es/) (una comunidad de profesionales en el área de sistemas) que contiene los resultados de una encuesta se sueldos realizada en 2020. La encuesta tiene algunso campos tipificados y otros campos de texto libre.

Queremos analizar los resultados de esta encuesta. Nos interesan especialmente aquellas respuestas vinculadas a profesionales que trabajan con datos y que poseen título universitario (para eliminar algunos casos con comportamiento atípico). Los objetivos de esta práctica serán:

1. Construir una variable que nos indique hace cuánto tiempo fue respondida la encuesta.

2. Encontrar el salario promedio para los trabajos vinculados a datos. Puede ser general o abierto por las distintas variantes.

3. Contabilizar cuantos lenguajes de programación maneja la persona.


In [2]:
sueldos = pd.read_csv("https://raw.githubusercontent.com/eea-uba/EEA-2020/master/Fuentes/encuesta_sueldos_sysarmy_1s2020.csv")
sueldos.head()

,timestamp,me_identifico,tengo,estoy_trabajando_en,donde_estas_trabajando,anos_de_experiencia,anos_en_la_empresa_actual,anos_en_el_puesto_actual,gente_a_cargo,nivel_de_estudios_alcanzado,...,sos_miembro_de_alguna_comunidad_it,cantidad_de_empleados,actividad_principal,la_recomendas_como_un_buen_lugar_para_trabajar,como_calificas_las_politicas_de_diversidad_e_inclusion,a_cuantos_kilometros_de_tu_casa_queda_la_oficina,beneficios_extra,habias_respondido_nuestra_encuesta_en_ediciones_anteriores,cuales_consideras_que_son_las_mejores_empresas_de_it_para_trabajar_en_este_momentoen_tu_ciudad,sueldo_dolarizado
0,1/31/2020 6:50:26,Hombre,32,Argentina,Ciudad Autónoma de Buenos Aires,10,3,1,0,Secundario,...,NaN,10001+,Producto basado en Software,8,10,6,"Capacitaciones y/o cursos, Comidas pagas / sub...",No,cognizant,False
1,1/28/2020 9:27:48,Hombre,30,Argentina,Ciudad Autónoma de Buenos Aires,3,3,3,0,Terciario,...,NaN,51-100,Otras industrias,7,3,35,Capacitaciones y/o cursos,No,mercadolibre,False
2,1/29/2020 16:54:29,Mujer,40,Argentina,Ciudad Autónoma de Buenos Aires,15,3,3,1,Universitario,...,NaN,11-50,Servicios / Consultoría de Software / Digital,8,8,30,"Abono de celular y/o Internet, Capacitaciones ...",No,NaN,False
3,2/1/2020 5:46:25,Mujer,36,Argentina,Ciudad Autónoma de Buenos Aires,10,3,3,0,Universitario,...,NaN,51-100,Otras industrias,7,5,4,Viáticos,No,NaN,False
4,2/1/2020 17:51:21,Hombre,29,Argentina,Ciudad Autónoma de Buenos Aires,10,2,2,0,Universitario,...,NaN,11-50,Otras industrias,5,8,7,Ninguna de las anteriores,No,NaN,False


In [3]:
# observamos faltantes y tipos de datos
sueldos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5982 entries, 0 to 5981
Data columns (total 53 columns):
 #   Column                                                                                          Non-Null Count  Dtype  
---  ------                                                                                          --------------  -----  
 0   timestamp                                                                                       5982 non-null   object 
 1   me_identifico                                                                                   5982 non-null   object 
 2   tengo                                                                                           5982 non-null   int64  
 3   estoy_trabajando_en                                                                             5982 non-null   object 
 4   donde_estas_trabajando                                                                          5982 non-null   object 
 5   anos_de_exper

In [4]:
# Antes de comenzar, filtramos solo los registros con educacion universitaria
sueldos = sueldos[sueldos.nivel_de_estudios_alcanzado == 'Universitario']

## 1) Fecha de encuesta

Pista: la función **datetime.datetime.now()** nos devuelve fecha y hora actuales.

In [5]:
sueldos.timestamp.head()

2    1/29/2020 16:54:29
3      2/1/2020 5:46:25
4     2/1/2020 17:51:21
5     2/1/2020 23:13:41
9    1/31/2020 15:35:31
Name: timestamp, dtype: object

In [6]:
# convertimos la variable datestamp a date
sueldos['fecha_de_respuesta'] = pd.to_datetime(sueldos.timestamp, format="%m/%d/%Y %H:%M:%S")
sueldos.fecha_de_respuesta.head()

2   2020-01-29 16:54:29
3   2020-02-01 05:46:25
4   2020-02-01 17:51:21
5   2020-02-01 23:13:41
9   2020-01-31 15:35:31
Name: fecha_de_respuesta, dtype: datetime64[ns]

In [7]:
# generamos la variable de antiguedad (diferencia con hoy)
today = dt.datetime.now()

sueldos['dias_desde_respuesta'] = (today - sueldos.fecha_de_respuesta)
sueldos.dias_desde_respuesta.head()

2   1604 days 04:55:51.392359
3   1601 days 16:03:55.392359
4   1601 days 03:58:59.392359
5   1600 days 22:36:39.392359
9   1602 days 06:14:49.392359
Name: dias_desde_respuesta, dtype: timedelta64[ns]

## 2) Trabajo con datos

In [8]:
sueldos.trabajo_de.unique()

array(['Abogada', 'Abogado', 'Account Manager-Sales', 'Adm',
       'Administración', 'Administrador de Infraestructura',
       'Administrativa', 'Administrativa contable',
       'Administrative Assistant', 'Administrativo',
       'Administrativo en farmacia', 'Analista',
       'Analista de Operaciones de Servicios',
       'Analista de prevención de lavado de activos',
       'Analista de Soporte', 'Analista Funciona ERP',
       'Functional Analyst', 'Developer', 'Analista SAP',
       'Analista Senior.', 'Analyst support', 'Anlytics Engineer',
       'Application Support', 'Applications Support Analyst', 'Architect',
       'Asistente de Sistemas', 'Assistant', 'Atención al cliente',
       'Atención al publico', 'Auditor de TI', 'Auditoria',
       'BI Analyst / Data Analyst', 'Data Scientist / Data Engineer',
       'DBA Especializado / Gurú', 'DBA', 'BI Sr. Manager',
       'Business Analyst', 'Cadete', 'Cajera', 'Cajero', 'Call center',
       'Captador de donaciones', 'Chef

In [9]:
# llevamos todo a minuscula
sueldos['trabajo_de'] = sueldos.trabajo_de.str.lower()

In [10]:
# creo una variable binaria para ver si trabaja con datos
sueldos['trabajo_con_datos'] = sueldos.trabajo_de.str.contains('data')

In [11]:
# Verificamos nuestro filtro
sueldos[sueldos['trabajo_con_datos'] == True].trabajo_de.unique()

array(['bi analyst / data analyst', 'data scientist / data engineer',
       'data entry', 'data entry and management',
       'lider data analyst qa - tester'], dtype=object)

In [12]:
# Filtramos los profesionales de datos
sueldos_data = sueldos[sueldos['trabajo_con_datos'] == True]

In [13]:
# Normalizamos el puesto de 'data entry' y 'data analyst
sueldos_data['trabajo_de'] = sueldos_data.trabajo_de.str.replace('data entry and management', 'data entry')
sueldos_data['trabajo_de'] = sueldos_data.trabajo_de.str.replace('lider data analyst qa - tester', 'bi analyst / data analyst')

In [14]:
sueldos_data.trabajo_de.unique()

array(['bi analyst / data analyst', 'data scientist / data engineer',
       'data entry'], dtype=object)

In [15]:
# Ahora vemos el salario promedio
np.mean(sueldos_data.salario_mensual_bruto_en_tu_moneda_local)

128527.74056603774

In [16]:
sueldos_data.groupby('trabajo_de').agg({'salario_mensual_bruto_en_tu_moneda_local': [np.mean, np.median]})

C:\Users\carlos\AppData\Local\Temp\ipykernel_14888\3497278532.py:1: FutureWarning: The provided callable <function mean at 0x00000249FEBE1B20> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  sueldos_data.groupby('trabajo_de').agg({'salario_mensual_bruto_en_tu_moneda_local': [np.mean, np.median]})
C:\Users\carlos\AppData\Local\Temp\ipykernel_14888\3497278532.py:1: FutureWarning: The provided callable <function median at 0x00000249FED09E40> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  sueldos_data.groupby('trabajo_de').agg({'salario_mensual_bruto_en_tu_moneda_local': [np.mean, np.median]})


salario_mensual_bruto_en_tu_moneda_local  \
                                                                   mean   
trabajo_de                                                                
bi analyst / data analyst                                 154215.290323   
data entry                                                 22000.000000   
data scientist / data engineer                             93967.267442   

                                         
                                 median  
trabajo_de                               
bi analyst / data analyst       61250.0  
data entry                      22000.0  
data scientist / data engineer  80000.0

## 3) Lenguajes de programación

In [17]:
sueldos_data.lenguajes_de_programacion

240                                    Java, SQL, VBA
242                                               SQL
244     .NET, C#, CSS, HTML, PHP, Python, R, SQL, VBA
245                  Assembler, C#, COBOL, HTML, Java
247                                       Python, SQL
                            ...                      
904                                               NaN
906                     HTML, Javascript, Python, SQL
908                                               SQL
912                                               SQL
3704                          HTML, Java, Python, SQL
Name: lenguajes_de_programacion, Length: 212, dtype: object

In [18]:
sueldos_data['lenguajes_de_programacion_split'] = sueldos_data.lenguajes_de_programacion.str.split(',')

In [19]:
sueldos_data['cantidad_lenguajes'] = sueldos_data.lenguajes_de_programacion_split.str.len()

In [20]:
sueldos_data.groupby('trabajo_de').agg({'cantidad_lenguajes': [np.nanmean, np.median]})

C:\Users\carlos\AppData\Local\Temp\ipykernel_14888\335890869.py:1: FutureWarning: The provided callable <function nanmean at 0x00000249FED113A0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  sueldos_data.groupby('trabajo_de').agg({'cantidad_lenguajes': [np.nanmean, np.median]})
C:\Users\carlos\AppData\Local\Temp\ipykernel_14888\335890869.py:1: FutureWarning: The provided callable <function median at 0x00000249FED09E40> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  sueldos_data.groupby('trabajo_de').agg({'cantidad_lenguajes': [np.nanmean, np.median]})


cantidad_lenguajes       
                                          nanmean median
trabajo_de                                              
bi analyst / data analyst                2.271845    2.0
data entry                                    NaN    NaN
data scientist / data engineer           3.059524    3.0